In [75]:
# Import libs
import pandas as pd
import geopandas as gpd
import folium
import folium.plugins as plgns
from tqdm import tqdm
import os
import numpy as np
from plotly import graph_objects as go

In [76]:
# Load the district ids and sensor data
# Aggregate data by district and by day

# Path
path_to_district = "./districts"
path_to_sensor_data = "./preprocessed_data/city/per_day"
district_data = {} # Use a dictionary to store the data by district

# Read the data by district and aggregate
for filename in tqdm(os.listdir(path_to_district), desc="Reading districts", position=0):
    district_id = pd.read_csv(os.path.join(path_to_district, filename))['detid']
    kreis_name = filename.replace('.csv','').replace(' ', '_')
    district_data[kreis_name] = []
    for sensor_id in tqdm(district_id, desc="Reading sensor data", position=1):
        try:
            sensor_df = pd.read_csv(os.path.join(path_to_sensor_data, sensor_id+".csv"))
            district_data[kreis_name].append(sensor_df)
        except:
            pass




Reading districts: 100%|██████████| 12/12 [00:03<00:00,  3.28it/s]


In [77]:
for key in tqdm(district_data.keys(), desc="Aggregating"):
    district_data[key] = pd.concat(district_data[key])
    district_data[key]['date'] = pd.to_datetime(district_data[key]['date'])
    district_data[key]['day'] = pd.to_datetime(district_data[key]['date']).dt.weekday
    # Only consider workdays
    district_data[key] = district_data[key][district_data[key]['day'] <=4]
    # Aggregate by date, use average occupancy and flow to represent a district
    district_data[key] = district_data[key].groupby(['date']).agg(occ_avg = pd.NamedAgg(column="occ_sum", aggfunc="mean"),
                                                                  flow_avg = pd.NamedAgg(column="flow_sum", aggfunc="mean"))
    # district_data[key]['color_occ'] = pd.cut(district_data[key]['occ_avg'], bins=10, labels=['#FFEBEB','#F8D2D4','#F2B9BE','#EBA1A8','#E58892','#DE6F7C','#D85766','#D13E50','#CB253A','#C50D24'])
    # district_data[key]['color_flow'] = pd.cut(district_data[key]['flow_avg'], bins=10, labels=['#FFEBEB','#F8D2D4','#F2B9BE','#EBA1A8','#E58892','#DE6F7C','#D85766','#D13E50','#CB253A','#C50D24'])

Aggregating: 100%|██████████| 12/12 [00:00<00:00, 14.21it/s]


In [88]:
for kreis in tqdm(district_data.keys(), "Drawing"):
    data = district_data[kreis]
    Kreis_id = kreis
    columns = data.columns
    n_plot = len(columns)
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
    fig = go.Figure()
    # print(data.index)

    for i in range(len(columns)):
        fig.add_trace(
            go.Scatter(
                x=data.index,
                y=data[columns[i]],
                name=columns[i],
                yaxis="y" if i==0 else f"y{i+1}"
            )
        )
    
    fig.update_traces(
        hoverinfo="x+y",
        line={"width": 0.5},
        mode="lines",
        showlegend=False
    )

    layout_dict = dict(
        title_text= Kreis_id + " info",
        title_x=0.5,
        title_font_family="Times New Roman",
        title_font_size = 20,
        xaxis=dict(
            tickfont_size=10,
            tickangle = 270,
            # showgrid = True,
            zeroline = True,
            showline = True,
            # showticklabels = True,
            # dtick="D1", #Change the x-axis ticks to be daily
            ),
    )
    
    for i in range(len(columns)):
        layout_dict['yaxis' if i==0 else f'yaxis{i+1}'] = dict(
            anchor="x",
            autorange=True,
            domain=[i/(n_plot+1) + i/(n_plot+1)/n_plot, (i+1)/(n_plot+1) + i/(n_plot+1)/n_plot],
            linecolor=colors[i],
            mirror=True,
            showline=True,
            side="right",
            tickfont={"color": colors[i]},
            tickmode="auto",
            titlefont={"color": colors[i]},
            type="linear",
            title=columns[i],
            zeroline=False
        )
    
    fig.update_layout(layout_dict)

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="7d", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ]))
    )

    # Save the fig
    fig.write_html(os.path.join("./figs", Kreis_id + ".html"))

Drawing: 100%|██████████| 12/12 [00:00<00:00, 19.23it/s]


In [79]:
print(district_data['Kreis_1'])

               occ_avg      flow_avg
date                                
2018-01-01  101.591135  68589.612157
2018-01-02   89.151427  63707.872457
2018-01-03  112.800506  78202.293676
2018-01-04  129.912065  85877.460023
2018-01-05  142.791629  95031.107073
...                ...           ...
2021-12-27  101.019937  78584.210394
2021-12-28   98.253292  78367.229614
2021-12-29   98.617099  78726.956285
2021-12-30  115.055617  85141.972262
2021-12-31  109.088229  83481.878846

[1045 rows x 2 columns]


In [80]:
# Load gis data
path_to_district_gis = './spatial/districts/'
path_to_sensor_gis = './spatial/loop_update_city/'

district_gdf = gpd.read_file(path_to_district_gis).to_crs("EPSG:4326")
district_gdf['bezeichnun'] = district_gdf['bezeichnun'].apply(lambda x: x.replace(' ', '_'))
district_gdf.drop(columns=['objid', 'name', 'entstehung'], inplace=True)
district_gdf.rename(columns={'bezeichnun': 'Kreis'}, inplace=True)
city_gdf = gpd.read_file(path_to_sensor_gis).to_crs("EPSG:4326")
city_geo_list = np.array([[point.xy[1][0], point.xy[0][0]] for point in city_gdf['geometry']])

In [81]:
print(district_gdf)

       Kreis                                           geometry
0    Kreis_6  POLYGON ((8.52602 47.40844, 8.52625 47.40855, ...
1   Kreis_11  POLYGON ((8.48438 47.42904, 8.48439 47.42909, ...
2   Kreis_12  POLYGON ((8.55353 47.39929, 8.55358 47.39935, ...
3   Kreis_10  POLYGON ((8.46802 47.41334, 8.46803 47.41334, ...
4    Kreis_4  POLYGON ((8.49762 47.38834, 8.49789 47.38876, ...
5    Kreis_1  POLYGON ((8.52922 47.36920, 8.52957 47.36944, ...
6    Kreis_9  POLYGON ((8.44802 47.38025, 8.44856 47.38027, ...
7    Kreis_5  POLYGON ((8.49837 47.39205, 8.49899 47.39205, ...
8    Kreis_7  POLYGON ((8.54866 47.37709, 8.54867 47.37708, ...
9    Kreis_3  POLYGON ((8.48343 47.35802, 8.48355 47.35820, ...
10   Kreis_2  POLYGON ((8.50065 47.33110, 8.50075 47.33138, ...
11   Kreis_8  POLYGON ((8.54214 47.36320, 8.54541 47.36410, ...


In [82]:
# Display time-varying map

start_location = np.mean(city_geo_list, axis=0).tolist()
map = folium.Map(location=start_location, tiles="OpenStreetMap", zoom_start=12)

# Add sensor positions
city_sensors = folium.FeatureGroup(name="city_sensors", show=True)

for sensor_position in city_geo_list:
    folium.CircleMarker(location=sensor_position, radius=5, color="orange").add_to(city_sensors)

city_sensors.add_to(map)

# Add district
for _, r in district_gdf.iterrows():
    geo = gpd.GeoSeries(r['geometry'])
    geo_j = geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'blue'})
    folium.Popup(r['Kreis']).add_to(geo_j)
    geo_j.add_to(map)

# Add heat map
# time_slider = plgns.TimeSliderChoropleth()

folium.LayerControl().add_to(map)
map